# Tekstiluokittelutehtävä

Kuten mainitsimme, keskitymme yksinkertaiseen tekstiluokittelutehtävään, joka perustuu **AG_NEWS**-aineistoon. Tehtävänä on luokitella uutisotsikot yhteen neljästä kategoriasta: Maailma, Urheilu, Liiketoiminta ja Tiede/Teknologia.

## Aineisto

Tämä aineisto sisältyy [`torchtext`](https://github.com/pytorch/text) -moduuliin, joten voimme käyttää sitä helposti.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Tässä `train_dataset` ja `test_dataset` sisältävät kokoelmia, jotka palauttavat parit, joissa on luokan numero (luokan numero) ja teksti vastaavasti, esimerkiksi:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Joten, tulostetaan ensimmäiset 10 uutta otsikkoa tietoaineistostamme:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Koska tietojoukot ovat iteraattoreita, jos haluamme käyttää dataa useita kertoja, meidän täytyy muuntaa se listaksi:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Tokenisointi

Nyt meidän täytyy muuntaa teksti **numeroiksi**, jotka voidaan esittää tensoreina. Jos haluamme sanatasoisen esityksen, meidän täytyy tehdä kaksi asiaa:
* käyttää **tokenisoijaa** jakamaan teksti **tokeneiksi**
* luoda näiden tokenien **sanasto**.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Sanaston avulla voimme helposti koodata tokenisoidun merkkijonon numerosarjaksi:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Bag of Words -tekstin esitys

Koska sanat välittävät merkitystä, joskus tekstin merkityksen voi päätellä pelkästään tarkastelemalla yksittäisiä sanoja, riippumatta niiden järjestyksestä lauseessa. Esimerkiksi uutisia luokitellessa sanat kuten *sää*, *lumi* viittaavat todennäköisesti *sääennusteeseen*, kun taas sanat kuten *osakkeet*, *dollari* liittyvät *talousuutisiin*.

**Bag of Words** (BoW) -vektoriesitys on yleisimmin käytetty perinteinen vektoriesitys. Jokainen sana on yhdistetty vektorin indeksiin, ja vektorin elementti sisältää sanan esiintymiskertojen määrän tietyssä dokumentissa.

![Kuva, joka näyttää, miten bag of words -vektoriesitys tallennetaan muistiin.](../../../../../translated_images/fi/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Voit myös ajatella BoW:n olevan summa kaikista yksittäisten sanojen yksi-hot-koodatuista vektoreista tekstissä.

Alla on esimerkki siitä, miten bag of words -esitys voidaan luoda Scikit Learn -python-kirjaston avulla:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

AG_NEWS-datasarjan vektoriedustuksesta bag-of-words-vektorin laskemiseksi voimme käyttää seuraavaa funktiota:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Huom:** Tässä käytämme globaalia `vocab_size`-muuttujaa määrittämään sanaston oletuskoko. Koska sanaston koko on usein melko suuri, voimme rajoittaa sanaston koon yleisimpiin sanoihin. Kokeile pienentää `vocab_size`-arvoa ja suorittaa alla oleva koodi, ja katso, miten se vaikuttaa tarkkuuteen. Voit odottaa jonkin verran tarkkuuden laskua, mutta ei dramaattista, paremman suorituskyvyn sijaan.


## BoW-luokittimen kouluttaminen

Nyt kun olemme oppineet rakentamaan Bag-of-Words-esityksen tekstistämme, koulutetaan sen päälle luokitin. Ensin meidän täytyy muuntaa datamme koulutusta varten siten, että kaikki paikkavektoriesitykset muutetaan Bag-of-Words-esitykseksi. Tämä voidaan tehdä antamalla `bowify`-funktio `collate_fn`-parametrina standardille torch `DataLoader`:ille:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Nyt määritellään yksinkertainen luokittelijaneuroverkko, joka sisältää yhden lineaarisen kerroksen. Syötevektorin koko on yhtä suuri kuin `vocab_size`, ja ulostulon koko vastaa luokkien määrää (4). Koska ratkaistavana on luokittelutehtävä, lopullinen aktivointifunktio on `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Nyt määritämme standardin PyTorch-koulutussilmukan. Koska datamme on melko suuri, opetustarkoituksiamme varten koulutamme vain yhden epookin ajan, ja joskus jopa vähemmän kuin yhden epookin ajan (määrittämällä `epoch_size`-parametrin voimme rajoittaa koulutusta). Raportoimme myös kertyneen koulutustarkkuuden koulutuksen aikana; raportoinnin tiheys määritetään käyttämällä `report_freq`-parametria.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGramit, TriGramit ja N-Gramit

Yksi bag of words -lähestymistavan rajoitus on, että jotkut sanat kuuluvat monisanaisiin ilmauksiin. Esimerkiksi sana 'hot dog' tarkoittaa täysin eri asiaa kuin sanat 'hot' ja 'dog' muissa yhteyksissä. Jos edustamme sanoja 'hot' ja 'dog' aina samoilla vektoreilla, se voi hämmentää malliamme.

Tämän ongelman ratkaisemiseksi käytetään usein **N-gram-edustuksia** dokumenttiluokittelumenetelmissä, joissa jokaisen sanan, kaksisanaisen tai kolmisanaisen ilmaisun esiintymistiheys on hyödyllinen ominaisuus luokittimien kouluttamisessa. Esimerkiksi bigram-edustuksessa lisäämme sanastoon kaikki sanaparit alkuperäisten sanojen lisäksi.

Alla on esimerkki siitä, kuinka bigram bag of words -edustus voidaan luoda Scikit Learnin avulla:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

N-gram-lähestymistavan suurin haittapuoli on, että sanaston koko alkaa kasvaa erittäin nopeasti. Käytännössä meidän täytyy yhdistää N-gram-esitys joihinkin dimensioiden vähentämistekniikoihin, kuten *upotuksiin*, joita käsittelemme seuraavassa osiossa.

Jotta voimme käyttää N-gram-esitystä **AG News** -aineistossamme, meidän täytyy luoda erityinen ngram-sanasto:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Voisimme käyttää samaa koodia kuin yllä kouluttaaksemme luokittelijan, mutta se olisi erittäin muistitehotonta. Seuraavassa osiossa koulutamme bigram-luokittelijan käyttämällä upotuksia.

> **Huom:** Voit jättää vain ne ngrammit, jotka esiintyvät tekstissä useammin kuin määritetty määrä kertoja. Tämä varmistaa, että harvoin esiintyvät bigrammit jätetään pois, ja pienentää merkittävästi ulottuvuuksia. Tämän saavuttamiseksi aseta `min_freq`-parametri korkeammaksi ja tarkkaile sanaston pituuden muutosta.


## Term Frequency Inverse Document Frequency TF-IDF

BoW-esityksessä sanojen esiintymiset painotetaan tasaisesti riippumatta itse sanasta. On kuitenkin selvää, että yleiset sanat, kuten *a*, *in* jne., ovat paljon vähemmän tärkeitä luokittelun kannalta kuin erikoistuneet termit. Itse asiassa useimmissa NLP-tehtävissä jotkut sanat ovat merkityksellisempiä kuin toiset.

**TF-IDF** tarkoittaa **term frequency–inverse document frequency** (termin esiintymistiheys–käänteinen dokumenttitiheys). Se on muunnelma bag of words -menetelmästä, jossa binäärisen 0/1-arvon sijaan, joka ilmaisee sanan esiintymisen dokumentissa, käytetään liukulukuarvoa, joka liittyy sanan esiintymistiheyteen korpuksessa.

Tarkemmin määriteltynä sanan $i$ paino $w_{ij}$ dokumentissa $j$ määritellään seuraavasti:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
missä
* $tf_{ij}$ on sanan $i$ esiintymiskertojen määrä dokumentissa $j$, eli BoW-arvo, jonka olemme aiemmin nähneet
* $N$ on kokoelman dokumenttien lukumäärä
* $df_i$ on niiden dokumenttien lukumäärä, jotka sisältävät sanan $i$ koko kokoelmassa

TF-IDF-arvo $w_{ij}$ kasvaa suhteessa siihen, kuinka monta kertaa sana esiintyy dokumentissa, ja sitä tasapainottaa niiden dokumenttien määrä korpuksessa, jotka sisältävät kyseisen sanan. Tämä auttaa huomioimaan sen, että jotkut sanat esiintyvät useammin kuin toiset. Esimerkiksi, jos sana esiintyy *jokaisessa* kokoelman dokumentissa, $df_i=N$, ja $w_{ij}=0$, jolloin nämä termit jätetään kokonaan huomiotta.

Voit helposti luoda tekstin TF-IDF-vektorisoinnin käyttämällä Scikit Learnia:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Johtopäätös

Vaikka TF-IDF-esitykset antavat sanoille painotuksia niiden esiintymistiheyden perusteella, ne eivät pysty ilmaisemaan merkitystä tai järjestystä. Kuten kuuluisa kielitieteilijä J. R. Firth totesi vuonna 1935: "Sanalla on aina täydellinen merkitys vain kontekstissaan, eikä merkityksen tutkimista ilman kontekstia voida ottaa vakavasti." Kurssin myöhemmässä vaiheessa opimme, kuinka tekstistä voidaan saada kontekstuaalista tietoa käyttämällä kielimallinnusta.



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulee pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskääntämistä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
